# Plant Seedling Classification

## Importing libraries

In [1]:
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import PIL
import warnings 
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras import regularizers
import keras
import tensorflow
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications import ResNet50

### Defining the path for test and train data

In [2]:
from pathlib import Path
data_dir_train = Path('/Users/siddhant/anaconda3/envs/train')
data_dir_test = Path('/Users/siddhant/anaconda3/envs/test')

In [3]:
import os
image_count_train = len(list(data_dir_train.glob('*/*.png')))
print(image_count_train)
image_count_test = len(list(data_dir_test.glob('*/*.png')))
print(image_count_test)

5549
768


### Load using keras.preprocessing

### Creating the dataset


In [4]:
# specifying the batch size and image dimensions
batch_size = 32
img_height = 120
img_width = 120
num_classes = 12

In [5]:
# creating train dataset
## Note use seed=123 while creating your dataset using tf.keras.preprocessing.image_dataset_from_directory
## Note, make sure your resize your images to the size img_height*img_width, while writting the dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_train, seed=123,
                                                               image_size=(img_height, img_width), batch_size=batch_size)

Found 5549 files belonging to 12 classes.


In [6]:
# creating validation dataset
## Note use seed=123 while creating your dataset using tf.keras.preprocessing.image_dataset_from_directory
## Note, make sure your resize your images to the size img_height*img_width, while writting the dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(data_dir_test, seed=123,
                                                             image_size=(img_height, img_width), batch_size=batch_size)

Found 768 files belonging to 12 classes.


In [7]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Using tranfer learnig to create the model

In [8]:
from tensorflow.keras.applications import ResNet50

resnet_weights_path = '/Users/siddhant/anaconda3/envs/computer_vision/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes,activation = 'softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 12)                24588     
                                                                 
Total params: 23612300 (90.07 MB)
Trainable params: 24588 (96.05 KB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________


In [10]:
epochs = 13
batch_size = 32
my_new_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

my_new_model.fit(train_ds, validation_data=val_ds, epochs=epochs, batch_size=batch_size)

Epoch 1/13
174/174 [==============================] - 68s 387ms/step - loss: 0.4876 - accuracy: 0.8425 - val_loss: 0.4052 - val_accuracy: 0.8750
Epoch 2/13
174/174 [==============================] - 70s 402ms/step - loss: 0.3759 - accuracy: 0.8809 - val_loss: 0.4602 - val_accuracy: 0.8294
Epoch 3/13
174/174 [==============================] - 69s 397ms/step - loss: 0.3160 - accuracy: 0.9036 - val_loss: 0.3113 - val_accuracy: 0.9128
Epoch 4/13
174/174 [==============================] - 70s 403ms/step - loss: 0.2740 - accuracy: 0.9184 - val_loss: 0.2873 - val_accuracy: 0.9128
Epoch 5/13
174/174 [==============================] - 73s 420ms/step - loss: 0.2397 - accuracy: 0.9301 - val_loss: 0.2369 - val_accuracy: 0.9375
Epoch 6/13
174/174 [==============================] - 71s 410ms/step - loss: 0.2115 - accuracy: 0.9420 - val_loss: 0.2390 - val_accuracy: 0.9258
Epoch 7/13
174/174 [==============================] - 71s 406ms/step - loss: 0.1884 - accuracy: 0.9515 - val_loss: 0.2089 - val_ac